# Radio ML Classification - CNN Approach

Solution to the RadioML classification challenge using CNNs.

In [1]:
# Loading Data

pathToDataset = "C:/Users/aksha/Documents/Jupyter Notebooks/radioml-classification/Datasets/Standard/RML2016.10a_dict.pkl"

# Extract the pickle file
import pickle
import numpy as np
Xd = pickle.load(open(pathToDataset,'rb'),encoding="bytes")
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

# Description of Data

The dataset has the size 220,000×2×128, which means that there are 220,000 entries, each consisting of an array of size 2 × 128. Each array represents the samples of about 128 µs of a received waveform sampled with approximately 106 samples/second, and it contains between 8 and 16 modulation symbols. Since the samples of the signal waveforms are complex-valued, they have been stored as real and imaginary parts, and therefore we have arrays of size 2 × 128 in the data set.

That is, each row is essentially, **a + ib**.

The labels of the downloaded dataset contain two parameters: the modulation technique used (one of [’8PSK’, ’AM-DSB’, ’AM-SSB’, ’BPSK’,’CPFSK’, ’GFSK’, ’PAM4’, ’QAM16’, ’QAM64’, ’QPSK’, ’WBFM’], so 11 possible modulation techniques), and the signal-to noise ratio (SNR) value (one of [−20, −18, −16, −14, −12, −10, −8, −6, −4, −2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18], so 20 possible SNR values). The SNR is a measure for the quality of the communication channel. The higher the SNR, the less “noisy” is the channel.

Each item of the list is essentially like **(b'Modulation Type',SNR Value)** [Use os.fsdecode to extract Modulation Type as string] 

In [2]:
# Import Necessary Packages
%matplotlib inline
import os
import random
import tensorflow.keras.utils
import tensorflow.keras.models as models
from tensorflow.keras.layers import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, LayerNormalization
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import numpy as np

# Data Processing

The goal of this stage is to split data into Training, Validation and Testing data and then process the available data and transform (if needed) it into relevant inputs to be fed into the CNN.

1. Split Dataset into Training, Validation and Testing Data : 110,000 samples for training and validation and 110,000 samples for testing.

In [3]:
#  into training and test sets of the form we can train/test on 
random.seed(777)
np.random.seed(777)

index = np.arange(0,220000)
random.shuffle(index)

trainIdx = index[0:110000]
testIdx = index[110000:220000]

trainX = X[trainIdx]


# Create Validation Data Set
indexVal = np.arange(0,110000)
random.shuffle(indexVal)

realTrainIdx = indexVal[0:99000] 
valIdx = indexVal[99000:110000]

# Actual Training Data
realTrainX = trainX[realTrainIdx]
X_train = np.expand_dims(realTrainX, axis=-1) # Important

# Actual Validation Data
validX = trainX[valIdx]
X_valid = np.expand_dims(validX, axis=-1) # Important

# Actual Testing Data
testX = X[testIdx]
X_test = np.expand_dims(testX, axis=-1) # Important

In [4]:
# One Hot Encode Labels
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(np.asarray(lbl)[:,0])
print(lb.classes_)
lbl_encoded=lb.transform(np.asarray(lbl)[:,0])
ytrain=lbl_encoded[trainIdx]

y_train = ytrain[realTrainIdx]
y_valid = ytrain[valIdx]
y_test=lbl_encoded[testIdx]

[b'8PSK' b'AM-DSB' b'AM-SSB' b'BPSK' b'CPFSK' b'GFSK' b'PAM4' b'QAM16'
 b'QAM64' b'QPSK' b'WBFM']


# Design Neural Network

In [5]:
# Network Parameters
dropoutRate = 0.6

# Structure
inpShape = (2,128,1) # Shape of Input Data
CNN1_numFilt = 256 # Number of Filters in CNN Layer 1
CNN1_kernSize = (1,3) # Kernel Size of CNN Layer 1

CNN2_numFilt = 256 # Number of Filters in CNN Layer 2
CNN2_kernSize = (2,3) # Kernel Size of CNN Layer 2

CNN3_numFilt = 80 # Number of Filters in CNN Layer 3
CNN3_kernSize = (1,3) # Kernel Size of CNN Layer 3

CNN4_numFilt = 80 # Number of Filters in CNN Layer 4
CNN4_kernSize = (1,3) # Kernel Size of CNN Layer 4

Dense1_numNeurons = 128 # Number of Nodes in the First Dense Layer
numOutput = 11 # Number of Output Nodes

# Weight Initialization
weightInit = 'glorot_uniform' # Xavier Initialization

# Activation Functions
activationHidden = 'relu'
activationOutput = 'softmax'

# Loss Function
lossFunction = 'categorical_crossentropy'

# Learning Algorithm
netOptimizer = 'adam'

# Callbacks
callbackList = [
        tensorflow.keras.callbacks.ModelCheckpoint('Data/CNN-12/CNN-12-Weights_best.h5', monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')]

# Construct Network
model = models.Sequential()
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_1'))
model.add(ZeroPadding2D(padding=(0,1),name='ZPad1'))
model.add(Convolution2D(filters=CNN1_numFilt, kernel_size=CNN1_kernSize, activation=activationHidden, name='Conv_1'))
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_2'))
model.add(Dropout(dropoutRate))
model.add(ZeroPadding2D(padding=(1,1),input_shape=inpShape,name='ZPad2'))
model.add(Convolution2D(filters=CNN2_numFilt, kernel_size=CNN2_kernSize, activation=activationHidden, name='Conv_2'))
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_3'))
model.add(Dropout(dropoutRate))
model.add(Convolution2D(filters=CNN3_numFilt, kernel_size=CNN3_kernSize, activation=activationHidden, name='Conv_3'))
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_4'))
model.add(Dropout(dropoutRate))
model.add(Convolution2D(filters=CNN4_numFilt, kernel_size=CNN4_kernSize, activation=activationHidden, name='Conv_4'))
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_5'))
model.add(Dropout(dropoutRate))
model.add(Flatten())
model.add(Dense(Dense1_numNeurons, activation=activationHidden))
model.add(BatchNormalization(axis=-1,input_shape=inpShape, name='Normalization_6'))
model.add(Dropout(dropoutRate))
model.add(Dense(numOutput, activation=activationOutput))
model.compile(loss=lossFunction, optimizer=netOptimizer,metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Normalization_1 (BatchNormal (None, 2, 128, 1)         4         
_________________________________________________________________
ZPad1 (ZeroPadding2D)        (None, 2, 130, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 2, 128, 256)       1024      
_________________________________________________________________
Normalization_2 (BatchNormal (None, 2, 128, 256)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 2, 128, 256)       0         
_________________________________________________________________
ZPad2 (ZeroPadding2D)        (None, 4, 130, 256)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 3, 128, 256)       3

# Train Network

In [6]:
numEpochs = 100
batchSize = 1024

In [7]:
history = model.fit(X_train,y_train,epochs=numEpochs,batch_size=batchSize,validation_data=(X_valid,y_valid),callbacks=callbackList,verbose=2)

Train on 99000 samples, validate on 11000 samples
Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[1024,80,3,124] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/Normalization_5/FusedBatchNormV3 (defined at <ipython-input-7-de2e952720ce>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_2908]

Function call stack:
distributed_function


In [ ]:
# Save History
np_loss_history = np.array(history.history["loss"])
np.save('Data/CNN-12/lossHist.npy',np_loss_history)

np_accu_history = np.array(history.history["categorical_accuracy"])
np.save('Data/CNN-12/accuHist.npy',np_accu_history)

np_val_loss_history = np.array(history.history["val_loss"])
np.save('Data/CNN-12/valLossHist.npy',np_val_loss_history)

np_val_accu_history = np.array(history.history["val_categorical_accuracy"])
np.save('Data/CNN-12/valAccuHist.npy',np_val_accu_history)

# Test/Evaluate Network

In [ ]:
# Re-load Best Weights
model.load_weights('Data/CNN-12/CNN-12-Weights_best.h5')

model.evaluate(X_test,y_test)

# Visualize Results

In [ ]:
# Load Details of History
lHist = np.load('Data/CNN-12/lossHist.npy')
aHist = np.load('Data/CNN-12/accuHist.npy')

vLHist = np.load('Data/CNN-12/valLossHist.npy')
vAHist = np.load('Data/CNN-12/valAccuHist.npy')

# Show loss curves 
plt.figure()
plt.title('CNN 12 - Training performance')
plt.plot(lHist, label='Training Loss')
plt.plot(vLHist, label='Validation Loss')
plt.xlabel('Epochs')
plt.xticks(np.arange(0,len(lHist)+1,2),np.arange(1,len(lHist)+1,2))
plt.ylabel('Loss')
plt.legend()

In [ ]:
# Function to Extract Test Data of Specific SNR
def extractTest(data,labels,labelsEncoded,testIndex,snr):
    testData = data[testIndex]
    labelArray = np.array([labels])
    testLabels = labelArray[:,testIdx,:]
    testLabelsEncoded = labelsEncoded[testIdx]
    
    idxOP = list()
    
    # Loop Through Label Array To Get Index of Specific SNR
    for i in range(0,testLabels.shape[1]):
        if testLabels[0,i,1].decode('ascii')==snr:
            idxOP.append(i)
    
    # Return Subset of Test Data and Corresponding Labels
    opTestData = np.expand_dims(testData[idxOP,:,:],axis=-1)
    opTestLabel = testLabelsEncoded[idxOP]
    
    return opTestData, opTestLabel

def plot_confusion_matrix(cm, titleAdd, title='CNN 11 - Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title+titleAdd)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Confusion Matrix Function
def prepConfMat(testData,testLabel,predTestLabel,mods,title):
    modString = list()
    for i in range(0,len(mods)):
        modString.append(mods[i].decode('ascii'))
    
    conf = np.zeros([len(mods),len(mods)])
    confnorm = np.zeros([len(mods),len(mods)])
    for i in range(0,testData.shape[0]):
        j = list(testLabel[i,:]).index(1)
        k = int(np.argmax(predTestLabel[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(mods)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    plot_confusion_matrix(confnorm, title, labels=modString)

In [ ]:
# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=1024)

prepConfMat(X_test,y_test,test_Y_hat,mods,' (All SNRs)')

In [ ]:
# Plot Confusion Matrix for Specific SNR
snr = '0'
title = ' (SNR = '+snr+')'
x_testSNR, y_TestSNR = extractTest(X,lbl,lbl_encoded,testIdx,snr)
y_hat_snr = model.predict(x_testSNR, batch_size=1024)
prepConfMat(x_testSNR,y_TestSNR,y_hat_snr,mods,title)

In [ ]:
# Get the test accuracy for different SNRs
acc = {}
acc_array=[]

snr_array=np.asarray(lbl)[:,1]
lb_temp = preprocessing.LabelBinarizer()
lb_temp.fit(snr_array)
temp_array=lb_temp.classes_
snr_label_array = []


snr_label_array.append(temp_array[6])
snr_label_array.append(temp_array[4])
snr_label_array.append(temp_array[3])
snr_label_array.append(temp_array[2])
snr_label_array.append(temp_array[1])
snr_label_array.append(temp_array[0])
snr_label_array.append(temp_array[9])
snr_label_array.append(temp_array[8])
snr_label_array.append(temp_array[7])
snr_label_array.append(temp_array[5])
snr_label_array.append(temp_array[10])
snr_label_array.append(temp_array[16])
snr_label_array.append(temp_array[17])
snr_label_array.append(temp_array[18])
snr_label_array.append(temp_array[19])
snr_label_array.append(temp_array[11])
snr_label_array.append(temp_array[12])
snr_label_array.append(temp_array[13])
snr_label_array.append(temp_array[14])
snr_label_array.append(temp_array[15])


#print(snr_label_array)
y_test_snr=snr_array[testIdx]



for snr in snr_label_array:
    test_X_i = X_test[np.where(y_test_snr==snr)]
    test_Y_i = y_test[np.where(y_test_snr==snr)]
    
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(mods),len(mods)])
    confnorm = np.zeros([len(mods),len(mods)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(mods)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    
    #plt.figure()
    #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print("Overall Accuracy: ", cor / (cor+ncor),"for SNR",snr)
    acc[snr] = 1.0*cor/(cor+ncor)
    acc_array.append(1.0*cor/(cor+ncor))

print("Random Guess Accuracy:",1/11)

In [ ]:
# Show loss curves 
plt.figure(figsize=(8, 6))
plt.title('CNN 12 - Accuracy vs SNRs')
plt.plot(np.arange(-20,20,2), acc_array)
plt.xlabel('SNR')
plt.xticks(np.arange(-20,20,2))
plt.ylabel('Class Accuracy')
plt.grid()
plt.show()

In [ ]:
accRes = np.array([acc_array])
np.save('Data/CNN-12/accResSNR.npy',accRes)